This notebook contains the commands to export selected visualizations as HTML. Just run the notebook to save the visualizations as interactive HTML maps. Use the [visualize](./visualize.ipynb) notebook if you want to learn how to create dataframes and visualizations from PostGIS data.

In [1]:
import sys
from ipygis import get_connection_url, QueryResult, generate_map
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from kepler_h3_config import config  # we may use our own custom visualization config

In [2]:
sql_url = get_connection_url(dbname='geoviz')
engine = create_engine(sql_url)
session = sessionmaker(bind=engine)()

sys.path.insert(0, '..')
from models import OSMPoint, OSMPolygon, FlickrPoint, GTFSStop, OSMAccessNode

The next phase will take a while. Here we actually create the dataframes in memory, from the whole database.

In [3]:
eating_query = session.query(OSMPoint).filter(OSMPoint.tags['amenity'].astext.in_(
    ['restaurant','bar','pub','biergarten','cafe','fast_food','food_court','ice_cream']
))
culture_query = session.query(OSMPoint).filter(OSMPoint.tags['amenity'].astext.in_(
    ['arts_centre','cinema','community_centre','conference_centre','events_venue','nightclub','theatre']
)|(OSMPoint.tags['tourism'].astext.in_(['museum','gallery'])))
shops_query = session.query(OSMPoint).filter(OSMPoint.tags.has_key('shop'))
photos_query = session.query(FlickrPoint)
stops_query = session.query(GTFSStop)
access_points_query = session.query(OSMAccessNode)

eating = QueryResult.create(eating_query, resolution=8, name='eating')
culture = QueryResult.create(culture_query, resolution=8, name='culture')
shops = QueryResult.create(shops_query, resolution=8, name='shops')
photographers = QueryResult.create(photos_query, resolution=8, name='photographers', group_by='properties.owner')
trips = QueryResult.create(stops_query, resolution=8, name='trips', plot='sum', column='properties.ntrips')
access = QueryResult.create(access_points_query, resolution=8, name='access', plot='mean', column='accessibilities.5')

Once the QueryResults exist, calculating the sum map from dataframes should be relatively fast.

In [4]:
results = [eating, culture, shops, photographers, trips, access]
columns = ['size', 'size', 'size', 'size', 'ntrips', '5']
weights = [1, 1, 1, 1, 1, -1]
#results = [eating, culture]
#columns = ['size', 'size']
#weights = [1, 1]

In [5]:
result_map = generate_map(results, 500, config=config, column=columns, weights=weights)
result_map.save_to_html()
result_map

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter
Map saved to keplergl_map.html!


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'hex8_eating', 'typ…